In [1]:
import warnings
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import datetime
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
import akshare
init_notebook_mode(connected=True)
sys.path.append("C:/dev/pycmqlib3/")
sys.path.append("C:/dev/pycmqlib3/misc_scripts/")
warnings.filterwarnings('ignore')
import misc
import data_handler as dh
import plotly_examples
import backtest
import copy
%matplotlib inline

ferrous_products_mkts = ['rb', 'hc', 'i', 'j', 'jm']
ferrous_mixed_mkts = ['ru', 'FG', 'ZC', 'SM', "SF"]
base_metal_mkts = ['cu', 'al', 'zn', 'pb', 'ni', 'sn']
precious_metal_mkts = ['au', 'ag']
ind_metal_mkts = ferrous_products_mkts + ferrous_mixed_mkts + base_metal_mkts  
petro_chem_mkts = ['l', 'pp', 'v', 'TA', 'MA', 'bu'] #, 'sc', 'fu', 'eg']
ind_all_mkts = ind_metal_mkts + petro_chem_mkts
ags_oil_mkts = ['m', 'RM', 'y', 'p', 'OI', 'a', 'c', 'cs'] #, 'b']
ags_soft_mkts = ['CF', 'SR', 'jd']#, 'AP', 'sp']
ags_all_mkts = ags_oil_mkts + ags_soft_mkts
eq_fut_mkts = ['IF', 'IH', 'IC']
bond_fut_mkts = ['T', 'TF']
fin_all_mkts = eq_fut_mkts + bond_fut_mkts
commod_all_mkts = ind_all_mkts + ags_all_mkts + precious_metal_mkts
all_markets = commod_all_mkts + fin_all_mkts

In [23]:
# field_list = ['open', 'high', 'low', 'close', 'volume', 'openInterest', 'contract', 'shift']

start_date = datetime.date(2016,1,1)
end_date = datetime.date(2020,6,11)

#sim_products = ['rb', 'hc', 'i', 'j', 'jm', 'ru', 'FG', 'ZC', 'OI', \
#                'l', 'pp', 'v', 'TA', 'MA', \
#                'a', 'm', 'RM', 'y', 'p', 'SR', 'CF', 'jd',\
#                'c', 'cs', 'cu', 'zn', 'al']
sim_products = ['jd']
need_shift = 1
freq = 'd'

args = {'roll_rule': '-20b', 'freq': freq, 'need_shift': need_shift}
ferrous_products_args = args
ferro_mixed_mkt_args = args
base_args = {'roll_rule': '-30b', 'freq': freq, 'need_shift': need_shift}
ni_args = {'roll_rule': '-30b', 'freq': freq, 'need_shift': need_shift}
eq_args = {'roll_rule': '1b', 'freq': freq, 'need_shift': need_shift}
bond_args = {'roll_rule': '-20b', 'freq': freq, 'need_shift': need_shift}
precious_args = {'roll_rule': '-20b', 'freq': freq, 'need_shift': need_shift}

df_list = []

for idx, asset in enumerate(sim_products):
    use_args = copy.copy(args)
    if asset in eq_fut_mkts:
        use_args = eq_args
    elif asset in ['cu', 'al', 'zn', 'pb', 'sn']:
        use_args = base_args
    elif asset in ['ni']:
        use_args = ni_args
    elif asset in bond_fut_mkts:
        use_args = bond_args
    elif asset in precious_metal_mkts:
        use_args = precious_args
    use_args['start_date'] = max(backtest.sim_start_dict[asset], start_date)
    use_args['end_date'] = end_date    
    for idy, nb in enumerate([1, 2]):
        use_args['n'] = nb
        print("loading mkt = %s, nb = %s, args = %s" % (asset, nb, use_args))
        df = misc.nearby(asset, **use_args)
        if freq == 'm':
            df = misc.cleanup_mindata(df, asset)
        if (idx == 0) and (idy == 0) and (freq == 'm'):
            df_list.append(df[['date', 'min_id']])
        xdf = df[field_list]
        xdf['expiry'] = xdf['contract'].apply(lambda x: misc.contract_expiry(x, misc.CHN_Holidays))
        xdf.columns = [(asset+str(nb), col) for col in xdf.columns]
        df_list.append(xdf)
all_df = pd.concat(df_list, axis = 1, sort = False).dropna()

loading mkt = jd, nb = 1, args = {'roll_rule': '-20b', 'freq': 'd', 'need_shift': 1, 'start_date': datetime.date(2016, 1, 1), 'end_date': datetime.date(2020, 6, 11), 'n': 1}
loading mkt = jd, nb = 2, args = {'roll_rule': '-20b', 'freq': 'd', 'need_shift': 1, 'start_date': datetime.date(2016, 1, 1), 'end_date': datetime.date(2020, 6, 11), 'n': 2}


In [10]:
bar_freq = 's1'

split_dict = {'s1': [300, 2100],
             's2': [300, 1500, 2100],
             's3': [300, 1500, 1930, 2100],
             's4': [300, 1500, 1630, 1930, 2100],}

xdf = all_df.copy()
xdf = dh.day_split1(xdf, split_dict[bar_freq])

In [11]:
print(xdf)

                           date  min_idx  min_id  (rb, open)  (rb, high)  \
datetime                                                                   
2019-11-01 21:00:00  2019-11-01        1    2059      2996.0      3016.0   
2019-11-04 21:00:00  2019-11-04        1    2059      3012.0      3022.0   
2019-11-05 21:00:00  2019-11-05        1    2059      3018.0      3070.0   
2019-11-06 21:00:00  2019-11-06        1    2059      3066.0      3102.0   
2019-11-07 21:00:00  2019-11-07        1    2059      3078.0      3093.0   
...                         ...      ...     ...         ...         ...   
2020-05-29 21:00:00  2020-05-29        1    2059      3499.0      3579.0   
2020-06-01 21:00:00  2020-06-01        1    2059      3579.0      3609.0   
2020-06-02 21:00:00  2020-06-02        1    2059      3570.0      3648.0   
2020-06-03 21:00:00  2020-06-03        1    2059      3640.0      3663.0   
2020-06-04 21:00:00  2020-06-04        1    2059      3633.0      3679.0   

           

In [20]:
df1 = pd.DataFrame(datetime.date(2020,5,1), index=range(10), columns = ['date1'])
df1['date2'] = datetime.date(2021,3,1)
df1['diff'] = df1.apply(lambda row: (row['date1']-row['date2']).days, axis=1)
print(df1)

        date1       date2  diff
0  2020-05-01  2021-03-01  -304
1  2020-05-01  2021-03-01  -304
2  2020-05-01  2021-03-01  -304
3  2020-05-01  2021-03-01  -304
4  2020-05-01  2021-03-01  -304
5  2020-05-01  2021-03-01  -304
6  2020-05-01  2021-03-01  -304
7  2020-05-01  2021-03-01  -304
8  2020-05-01  2021-03-01  -304
9  2020-05-01  2021-03-01  -304
